# Environmental setting

In [ ]:
!pip install -q ultralytics albumentations opencv-python

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from ultralytics import YOLO
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
import pandas as pd
from PIL import Image
import time
import shutil



In [ ]:
shutil.copytree(
    "/kaggle/input/vietnamses-traffic-sign-detection-augmentaion/dataset",
    "/kaggle/working/dataset"
)

## File path

In [ ]:
"""
    Liệt kê 10 file trong 1 folder 
"""

def list_10_file(input_dir):
    for dirname, _, filenames in os.walk(input_dir):
        if filenames: # nếu thư mục này có file
            print(f'Folder: {dirname}')
            for file in filenames[:10]:
                print(f'File: {file}')
        print() # xuống dòng

input_dir = '/kaggle/input'
list_10_file(input_dir)

In [ ]:
# faster_RCNN         
faster_RCNN_model = "/kaggle/input/bestmodel_faster_e-cnn/pytorch/default/1/faster_rcnn_model_epoch_8.pth"
coco_ann_path = "/kaggle/input/vietnam-traffic-sign-coco/faster_rcnn_coco_dataset/annotations/instances_test.json"
coco_img_dir = "/kaggle/input/vietnam-traffic-sign-coco/faster_rcnn_coco_dataset/images/test"


# YOLOv8
YOLOv8_model = "/kaggle/input/yolov8_epochs100/other/yolo_epochs100/1/last.pt"
yolov8_data_yaml = "/kaggle/working/dataset/data.yaml"




# Evaluate

In [ ]:

img_dir = "/kaggle/input/vietnam-traffic-sign-coco/faster_rcnn_coco_dataset/images/test"
bad_images = []

for file in os.listdir(img_dir):
    path = os.path.join(img_dir, file)
    try:
        with Image.open(path) as img:
            img.verify()  # kiểm tra ảnh có bị hỏng
    except Exception as e:
        print(f"[LỖI] {file}: {e}")
        bad_images.append(file)

print(f"\nTổng số ảnh lỗi: {len(bad_images)}")


### Function mAP, precicion, recall

In [ ]:
def load_faster_rcnn_model(model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = fasterrcnn_resnet50_fpn(pretrained=False)

    # Load checkpoint để lấy đúng số class
    checkpoint = torch.load(model_path, map_location=device)
    num_classes = checkpoint['roi_heads.box_predictor.cls_score.weight'].shape[0]

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    model.load_state_dict(checkpoint)
    model.to(device).eval()
    return model, num_classes


In [ ]:
def evaluate_faster_rcnn(model_path, ann_path, img_dir, score_thresh=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model, num_classes = load_faster_rcnn_model(model_path)

    coco = COCO(ann_path)
    results = []

    for img_id in tqdm(coco.getImgIds(), desc="Faster R-CNN evaluating"):
        img_info = coco.loadImgs(img_id)[0]
        file_name = os.path.basename(img_info['file_name'])  
        img_path = os.path.join(img_dir, file_name)


        # Đọc ảnh an toàn
        try:
            img = cv2.imread(img_path)
            if img is None:
                raise ValueError("cv2.imread trả về None")
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            try:
                img = Image.open(img_path).convert("RGB")
                img = np.array(img)
            except Exception as e:
                print(f"[ERROR] Không thể đọc ảnh {img_path}: {e}")
                continue

        img_tensor = torch.tensor(img / 255., dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(img_tensor)[0]

        for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
            if score < score_thresh:
                continue
            x1, y1, x2, y2 = box.tolist()
            results.append({
                "image_id": img_id,
                "category_id": int(label),
                "bbox": [x1, y1, x2 - x1, y2 - y1],
                "score": float(score)
            })

    # Lưu file kết quả
    with open("fasterrcnn_results.json", "w") as f:
        json.dump(results, f)

    # Đánh giá bằng COCO
    coco_dt = coco.loadRes("fasterrcnn_results.json")
    coco_eval = COCOeval(coco, coco_dt, iouType="bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # Trả về các chỉ số chính
    stats = coco_eval.stats
    return {
        "Model": "Faster R-CNN",
        "mAP@0.5": round(stats[1], 4),
        "mAP@0.5:0.95": round(stats[0], 4),
        "Recall": round(stats[8], 4),
        "Precision": round(stats[1], 4)  # dùng tạm AP50 làm Precision
    }


In [ ]:
def evaluate_yolov8(model_path, data_yaml):
    model = YOLO(model_path)
    metrics = model.val(data=data_yaml, split="test")

    return {
        "Model": "YOLOv8",
        "mAP@0.5": round(metrics.box.map50, 4),
        "mAP@0.5:0.95": round(metrics.box.map, 4),
        "Recall": round(metrics.box.mr, 4),
        "Precision": round(metrics.box.mp, 4)
    }


## mAP

In [ ]:
# Đánh giá
metrics_frcnn = evaluate_faster_rcnn(faster_RCNN_model, coco_ann_path, coco_img_dir)

metrics_yolov8 = evaluate_yolov8(YOLOv8_model, yolov8_data_yaml)

# So sánh bảng
df = pd.DataFrame([metrics_frcnn, metrics_yolov8])
display(df)

## Inference, FPS, Model Size, Parameters

### Function caculator FPS, Model Size, Parameters

In [ ]:
def evaluate_model_stats(model_path, model_type="yolov8", device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Đo kích thước mô hình
    model_size_mb = round(os.path.getsize(model_path) / (1024 * 1024), 2)

    # Load model
    if model_type == "yolov8":
        model = YOLO(model_path)
        model_f = model.model.fuse().to(device)
    elif model_type == "fasterrcnn":
        model = torch.load(model_path, map_location=device)
        model.eval().to(device)
        model_f = model
    else:
        raise ValueError("model_type must be either 'yolov8' or 'fasterrcnn'")

    # Dummy input image
    dummy_img = torch.randn(1, 3, 640, 640).to(device)

    # Đo inference time
    with torch.no_grad():
        start_time = time.time()
        for _ in range(10):
            if model_type == "yolov8":
                model(dummy_img)
            else:
                model([dummy_img[0]])  # Faster R-CNN expects list
        end_time = time.time()

    inference_time = (end_time - start_time) / 10  # time per image
    fps = round(1 / inference_time, 2)

    # Tham số mô hình
    num_params = sum(p.numel() for p in model_f.parameters())
    num_params_mil = round(num_params / 1e6, 2)

    return {
        "Model": model_type.upper(),
        "Model Size (MB)": model_size_mb,
        "Parameters (M)": num_params_mil,
        "Inference Time (s)": round(inference_time, 4),
        "FPS": fps
    }


In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def evaluate_model_stats(model_path, model_type="yolov8", device='cuda' if torch.cuda.is_available() else 'cpu'):
    model_size_mb = round(os.path.getsize(model_path) / (1024 * 1024), 2)

    if model_type == "yolov8":
        model = YOLO(model_path)
        model_f = model.model.fuse().to(device)
    elif model_type == "fasterrcnn":
        # Load checkpoint
        checkpoint = torch.load(model_path, map_location=device)

        # Lấy số class từ checkpoint
        num_classes = checkpoint['roi_heads.box_predictor.cls_score.weight'].shape[0]

        # Tạo kiến trúc model
        model = fasterrcnn_resnet50_fpn(pretrained=False)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

        # Load tham số
        model.load_state_dict(checkpoint)
        model.to(device).eval()

        model_f = model
    else:
        raise ValueError("model_type must be either 'yolov8' or 'fasterrcnn'")

    # Dummy input
    dummy_img = torch.randn(1, 3, 640, 640).to(device)

    with torch.no_grad():
        start_time = time.time()
        for _ in range(10):
            if model_type == "yolov8":
                model(dummy_img)
            else:
                model([dummy_img[0]])  # Faster R-CNN expects list
        end_time = time.time()

    inference_time = (end_time - start_time) / 10
    fps = round(1 / inference_time, 2)

    num_params = sum(p.numel() for p in model_f.parameters())
    num_params_mil = round(num_params / 1e6, 2)

    return {
        "Model": model_type.upper(),
        "Model Size (MB)": model_size_mb,
        "Parameters (M)": num_params_mil,
        "Inference Time (s)": round(inference_time, 4),
        "FPS": fps
    }


In [ ]:

frcnn_stats = evaluate_model_stats(faster_RCNN_model, model_type="fasterrcnn")
yolov8_stats = evaluate_model_stats(YOLOv8_model, model_type="yolov8")

# So sánh 2 mô hình dưới dạng bảng
df_stats = pd.DataFrame([frcnn_stats, yolov8_stats])
print(df_stats)


# Comment

aa